<a href="https://colab.research.google.com/github/Ramprabu95/AI_based_image_compression/blob/main/Learning_convolutional_Networks_for_Content_weighted_Image_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Convolutional Networks for Content Weighted Image Compression

# **Introduction:**


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

In [13]:
device = torch.device('cuda')
# Definition of convolution layer
def conv(ni, nf, kernal_size=3, stride=1, padding=1, **kwargs):
    _conv = nn.Conv2d(ni, nf, kernel_size=kernal_size,stride=stride,padding=padding, **kwargs)
    return _conv

# Definition of a residual block
class ResBlock(nn.Module):
    def __init__(self, ni, nh=128):
        super().__init__()
        self.conv1 = conv(ni, nh)
        self.conv2 = conv(nh, ni)
    def forward(self, x):
        return x  + self.conv2(F.relu(self.conv1(x)))

# Definition of Relu block
relu = nn.ReLU()

# Definition of encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.return_imp_map = return_imp_map
        #
        self.step1 = nn.Sequential(conv(ni=3, nf=128, kernal_size=8, stride=4, padding=2), relu,
                                   ResBlock(nh=128), relu,
                                   conv(ni=128, nf=256, kernal_size=4, stride=2, padding=1), relu,
                                   ResBlock(nh=256), relu,
                                   ResBlock(nh=256), relu)
    def forward(self,x):
        step1 = self.step1(x)
        return(step1)

# Function that does the binarizing action
class Binarizing_function(torch.autograd.Function):
  @staticmethod
  def forward(ctx, i):
    return (i>0.5).float()
  
  @staticmethod
  def backward(ctx,grad_output):
    return grad_output

def binary_value(x):
  return Binarizing_function.apply(x)

class Bin(nn.Module):
  def __init__(self,func):
    super().__init__()
    self.func = func

  def forward(self,x):
    return self.func(x)

# Definition of binarize 
class BinarizerOrImportanceMap(nn.Module):
  def __init__(self):
    super().__init__(self,return_imp_map=False)

    # Layer that contains the binarizer
    self.step2 = nn.Sequential(Encoder(),
                               conv(ni=256,nf=64,kernal_size=3,stride=1,padding=1),
                               nn.Sigmoid(),
                               Bin(binary_value))
    
    # Layer that contains the importance map 
    self.importance_map = nn.Sequential(Encoder(),
                                        conv(256,128),
                                        relu,
                                        conv(128,128),
                                        relu,
                                        conv(128,1),
                                        nn.Sigmoid())
    
    def forward(self,x):
      if self.return_imp_map:return self.importance_map(x)
      else: return self.step2(x)

# Quantizer of the importance map
class Quantizing(torch.autograd.Function):
  @staticmethod
  def forward(ctx, i):
    p = i.clone()
    L = 16
    for l in range(L):
      p[(p>=1/L)*(p<(l+1)/L)] = l
    return p
  @staticmethod
  def backward(ctx, grad_output):
    return grad_output
  
def quantize_values(x):
  return Quantizing.apply(x)

class Masking(torch.autograd.Function):
  @staticmethod
  def forward(ctx,i):
    N,_,H,W = i.shape
    n = 64 # as per the paper
    L = 16 # as per the paper
    mask = torch.zeros(n,N*H*W).to(device)
    q_important = i
    q_important_flat = q_important.view(1, N*H*W)
    for index in range(n):
      mask[index,:] = torch.where(index< n/L)*q_important_flat,torch.Tensor([1]).to(device),torch.Tensor([0]).to(device)
    mask = mask.view(n,N,H,W).permute((0,1,2,3))
    return mask
  @staticmethod
  def backward(ctx, grad_output):
     N,_,H,W = grad_output.shape
     if grad_output.is_cuda: return torch.ones(N,1,H,W).cuda()
     else: return torch.ones(N,1,H,W)

# function to generate mask
def generateMask(x):
  return Masking.apply(x)
  

# A decoder is to be defined which is a reverse of the encoder generated

class BitstoImage(torch.nn.Module):
  
  def __init__(self,blocksize):
    super().__init__()
    self.blocksize = blocksize
  
  def forward(self,x):
    N,C,H,W = x.size()
    x = x.view(N, self.blocksize, self.blocksize,C // (self.blocksize ** 2), H, W)
    x = x.permute(0,3,4,1,5,2).contiguous()
    x = x.view(N, C // (self.blocksize ** 2), H*self.blocksize, W*self.blocksize)
    return x

# Decoder design for rebulding the encoded data
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    # the convolution layers of a decoder is the reverse of the encoder
    self.decoder = nn.Sequential(conv(ni=64,nf=512,kernal_size=1,stride=1,padding=0),
                                 relu,
                                 ResBlock(512),
                                 relu,
                                 ResBlock(512),
                                 relu,
                                 BitstoImage(2),
                                 conv(ni=128, nf=256),
                                 relu,
                                 ResBlock(256),
                                 relu,
                                 BitstoImage(4),
                                 conv(ni=16,nf=32),
                                 relu,
                                 conv(ni=32,nf=3))
    def forward(self,x):
      return self.decoder(x)
  




